<h3 style='color:blue' align='center'>Handling imbalanced data in customer churn prediction</h3>

# Loading Data

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("/content/customer_churn.csv")
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4000,0420-HLGXF,Female,1,No,No,39,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.75,4036,No
2275,7180-PISOG,Male,0,Yes,Yes,1,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,74.50,74.5,Yes
2820,3251-YMVWZ,Male,0,No,No,53,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Bank transfer (automatic),24.05,1301.9,No
4305,0559-CKHUS,Female,0,Yes,No,29,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check,19.55,521.8,No
6239,0665-XHDJU,Male,0,Yes,No,72,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Electronic check,99.40,7285.7,No


# Data Preprocessing

In [4]:
df.drop('customerID',axis='columns',inplace=True)

In [5]:
pd.to_numeric(df.TotalCharges,errors='coerce').isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
7038    False
7039    False
7040    False
7041    False
7042    False
Name: TotalCharges, Length: 7043, dtype: bool

In [6]:
df[pd.to_numeric(df.TotalCharges,errors='coerce').isnull()]

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


In [7]:
df1 = df[df.TotalCharges!=' ']

In [8]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

In [9]:
def print_unique_col_values(df):
       for column in df:
            if df[column].dtypes=='object':
                print(f'{column}: {df[column].unique()}') 
print_unique_col_values(df1)

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']


In [10]:
df1.replace('No internet service','No',inplace=True)
df1.replace('No phone service','No',inplace=True)
print_unique_col_values(df1)

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes']
OnlineBackup: ['Yes' 'No']
DeviceProtection: ['No' 'Yes']
TechSupport: ['No' 'Yes']
StreamingTV: ['No' 'Yes']
StreamingMovies: ['No' 'Yes']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']


In [11]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
for col in yes_no_columns:
    df1[col].replace({'Yes': 1,'No': 0},inplace=True)

In [12]:
for col in df1:
    print(f'{col}: {df1[col].unique()}') 

gender: ['Female' 'Male']
SeniorCitizen: [0 1]
Partner: [1 0]
Dependents: [0 1]
tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService: [0 1]
MultipleLines: [0 1]
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: [0 1]
OnlineBackup: [1 0]
DeviceProtection: [0 1]
TechSupport: [0 1]
StreamingTV: [0 1]
StreamingMovies: [0 1]
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: [1 0]
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges: [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges: [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn: [0 1]


In [13]:
df1['gender'].replace({'Female':1,'Male':0},inplace=True)

In [14]:
df1.gender.unique()

array([1, 0])

In [15]:
df2 = pd.get_dummies(data=df1, columns=['InternetService','Contract','PaymentMethod'])
df2.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

In [16]:
df2.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
3114,0,0,0,0,5,1,0,0,0,1,1,1,1,0,75.15,392.65,0,1,0,0,1,0,0,1,0,0,0
2208,1,1,1,0,1,1,1,0,0,1,0,1,1,1,100.80,100.80,1,0,1,0,1,0,0,0,0,1,0
5153,0,1,1,0,41,1,0,0,1,0,0,1,1,1,94.55,3851.45,0,0,1,0,1,0,0,0,0,1,0
842,1,0,1,1,12,1,0,0,0,0,0,0,0,0,19.80,229.60,1,0,0,1,0,1,0,1,0,0,0
564,0,0,0,0,7,1,0,0,0,1,1,1,0,0,64.30,445.95,0,1,0,0,1,0,0,1,0,0,0


In [17]:
df2.dtypes

gender                                       int64
SeniorCitizen                                int64
Partner                                      int64
Dependents                                   int64
tenure                                       int64
PhoneService                                 int64
MultipleLines                                int64
OnlineSecurity                               int64
OnlineBackup                                 int64
DeviceProtection                             int64
TechSupport                                  int64
StreamingTV                                  int64
StreamingMovies                              int64
PaperlessBilling                             int64
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int64
InternetService_DSL                          uint8
InternetService_Fiber optic                  uint8
InternetService_No             

In [18]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale])

In [19]:
for col in df2:
    print(f'{col}: {df2[col].unique()}')

gender: [1 0]
SeniorCitizen: [0 1]
Partner: [1 0]
Dependents: [0 1]
tenure: [0.         0.46478873 0.01408451 0.61971831 0.09859155 0.29577465
 0.12676056 0.38028169 0.85915493 0.16901408 0.21126761 0.8028169
 0.67605634 0.33802817 0.95774648 0.71830986 0.98591549 0.28169014
 0.15492958 0.4084507  0.64788732 1.         0.22535211 0.36619718
 0.05633803 0.63380282 0.14084507 0.97183099 0.87323944 0.5915493
 0.1971831  0.83098592 0.23943662 0.91549296 0.11267606 0.02816901
 0.42253521 0.69014085 0.88732394 0.77464789 0.08450704 0.57746479
 0.47887324 0.66197183 0.3943662  0.90140845 0.52112676 0.94366197
 0.43661972 0.76056338 0.50704225 0.49295775 0.56338028 0.07042254
 0.04225352 0.45070423 0.92957746 0.30985915 0.78873239 0.84507042
 0.18309859 0.26760563 0.73239437 0.54929577 0.81690141 0.32394366
 0.6056338  0.25352113 0.74647887 0.70422535 0.35211268 0.53521127]
PhoneService: [0 1]
MultipleLines: [0 1]
OnlineSecurity: [0 1]
OnlineBackup: [1 0]
DeviceProtection: [0 1]
TechSupport: [

# Train test split

In [20]:
X = df2.drop('Churn',axis='columns')
y = testLabels = df2.Churn.astype(np.float32)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [21]:
y_train.value_counts()

0.0    4130
1.0    1495
Name: Churn, dtype: int64

In [22]:
y.value_counts()

0.0    5163
1.0    1869
Name: Churn, dtype: int64

# Build a model (ANN) in tensorflow/keras

In [23]:
from tensorflow_addons import losses
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [24]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(26, input_dim=26, activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

In [25]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
176/176 [==============================] - 0s 2ms/step - loss: 0.5119 - accuracy: 0.7424
Epoch 2/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4359 - accuracy: 0.7861
Epoch 3/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4245 - accuracy: 0.7947
Epoch 4/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4182 - accuracy: 0.7998
Epoch 5/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4148 - accuracy: 0.7995
Epoch 6/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4121 - accuracy: 0.8004
Epoch 7/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4097 - accuracy: 0.8020
Epoch 8/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4095 - accuracy: 0.8016
Epoch 9/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4067 - accuracy: 0.8052
Epoch 10/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4060 - accura

# Handling Imbalanced Data - Mitigating Skewdness of Data

## Method 1 - Undersampling

In [26]:
# Class count
count_class_0, count_class_1 = df1.Churn.value_counts()

# Divide by class
df_class_0 = df2[df2['Churn'] == 0]
df_class_1 = df2[df2['Churn'] == 1]

# Undersample 0-class and concat the DataFrames of both class
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Churn.value_counts())

Random under-sampling:
1    1869
0    1869
Name: Churn, dtype: int64


In [27]:
X = df_test_under.drop('Churn',axis='columns')
y = df_test_under['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [28]:
y_train.value_counts()

1    1495
0    1495
Name: Churn, dtype: int64

In [29]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6457 - accuracy: 0.6361
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5602 - accuracy: 0.7231
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5193 - accuracy: 0.7465
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5023 - accuracy: 0.7559
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4955 - accuracy: 0.7615
Epoch 6/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4892 - accuracy: 0.7635
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4886 - accuracy: 0.7612
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4847 - accuracy: 0.7599
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4799 - accuracy: 0.7639
Epoch 10/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4782 - accuracy: 0.7702
Epoch 11/

## Method 2 - Oversampling

In [30]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Churn.value_counts())

Random over-sampling:
1    5163
0    5163
Name: Churn, dtype: int64


In [31]:
X = df_test_over.drop('Churn',axis='columns')
y = df_test_over['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [32]:
y_train.value_counts()

1    4130
0    4130
Name: Churn, dtype: int64

In [33]:
loss = keras.losses.BinaryCrossentropy()
weights = -1
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 2ms/step - loss: 0.5586 - accuracy: 0.7123
Epoch 2/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4982 - accuracy: 0.7569
Epoch 3/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4910 - accuracy: 0.7565
Epoch 4/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4859 - accuracy: 0.7636
Epoch 5/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4829 - accuracy: 0.7637
Epoch 6/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4787 - accuracy: 0.7676
Epoch 7/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4764 - accuracy: 0.7699
Epoch 8/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4735 - accuracy: 0.7685
Epoch 9/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4720 - accuracy: 0.7700
Epoch 10/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4686 - accura

## Method 3 - SMOTE

In [34]:
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

In [36]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_sample(X, y)



In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [39]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 2ms/step - loss: 0.5454 - accuracy: 0.7396
Epoch 2/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4828 - accuracy: 0.7656
Epoch 3/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4733 - accuracy: 0.7732
Epoch 4/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4679 - accuracy: 0.7754
Epoch 5/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4636 - accuracy: 0.7795
Epoch 6/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4591 - accuracy: 0.7799
Epoch 7/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4554 - accuracy: 0.7844
Epoch 8/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4523 - accuracy: 0.7838
Epoch 9/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4495 - accuracy: 0.7845
Epoch 10/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4466 - accura

## Method 4 - Use of Ensemble with undersampling

In [40]:
df2.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [41]:
# Regain Original features and labels
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [43]:
y_train.value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [44]:
df3 = X_train.copy()
df3['Churn'] = y_train

In [45]:
df3_class0 = df3[df3.Churn==0]
df3_class1 = df3[df3.Churn==1]

In [46]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis='columns')
    y_train = df_train.Churn
    return X_train, y_train    

In [47]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6175 - accuracy: 0.6722
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5089 - accuracy: 0.7562
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4936 - accuracy: 0.7645
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4871 - accuracy: 0.7732
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4832 - accuracy: 0.7692
Epoch 6/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4817 - accuracy: 0.7692
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4787 - accuracy: 0.7689
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4774 - accuracy: 0.7706
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4753 - accuracy: 0.7739
Epoch 10/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4739 - accuracy: 0.7699
Epoch 11/

In [48]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6181 - accuracy: 0.6669
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5141 - accuracy: 0.7518
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4971 - accuracy: 0.7575
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4893 - accuracy: 0.7639
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4842 - accuracy: 0.7666
Epoch 6/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4802 - accuracy: 0.7699
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4773 - accuracy: 0.7666
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4763 - accuracy: 0.7686
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4735 - accuracy: 0.7689
Epoch 10/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4701 - accuracy: 0.7749
Epoch 11/

In [49]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
83/83 [==============================] - 0s 2ms/step - loss: 0.6014 - accuracy: 0.6869
Epoch 2/100
83/83 [==============================] - 0s 2ms/step - loss: 0.5149 - accuracy: 0.7541
Epoch 3/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4910 - accuracy: 0.7696
Epoch 4/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4828 - accuracy: 0.7738
Epoch 5/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4774 - accuracy: 0.7776
Epoch 6/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4719 - accuracy: 0.7780
Epoch 7/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4691 - accuracy: 0.7791
Epoch 8/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4668 - accuracy: 0.7829
Epoch 9/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4637 - accuracy: 0.7825
Epoch 10/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4620 - accuracy: 0.7810
Epoch 11/

In [50]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones>1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [51]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.89      0.68      0.77      1033
           1       0.47      0.78      0.59       374

    accuracy                           0.71      1407
   macro avg       0.68      0.73      0.68      1407
weighted avg       0.78      0.71      0.72      1407

